<a href="https://colab.research.google.com/github/kla55/langchain-llm/blob/main/langchain_RAG_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain openai sentence-transformers faiss-cpu

In [2]:
pip install -U langchain-community

In [11]:
pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 27.4 MB/s eta 0:00:00


In [12]:
from langchain.vectorstores import FAISS#, Pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
from pinecone import Pinecone, ServerlessSpec
#from langchain.vectorstores import Pinecone
import numpy as np
import os


In [14]:
# This is a pre-trained model designed for generating dense embeddings (numerical representations) for text.
# It's commonly used for tasks like semantic similarity, clustering, and retrieval.
embedding_model_name = 'sentence-transformers/all-MiniLM-L6-v2'

# A class that leverages sentence_transformers models from Hugging Face to generate embeddings for text
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

# This class provides an abstraction over Hugging Face models to generate embeddings from input text.
# When you pass text to this instance, it converts the text into a vector representation (a high-dimensional numerical format) that captures the semantic meaning of the text.
llm = HuggingFaceHub(repo_id="gpt2", huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN, model_kwargs={"temperature": 0.7, "max_length": 200})

<ipython-input-14-a82ecb3591b4>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.

In [15]:
# Use the model to generate a response
prompt = "How do you walk a dog?"
response = llm(prompt)
print(response)

<ipython-input-15-4cacd9bc0f1e>:3: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


How do you walk a dog? Some people may think that walking a dog is as simple as putting a leash on and heading out the door, but there are some tips and tricks that can make your walk more enjoyable for both you and your dog. Here are some steps to help you:

1. **Start with the Basics:**
   - **Leash Training:** Before you start walking, your dog should be comfortable with a leash. Start by holding the leash and letting your dog get used to the feeling. Reward them with treats or praise when they stay calm.
   - **Basic Commands:** Teach your dog basic commands like "sit," "stay," and "come." These will be helpful when you're out and about.

2. **Choose the Right Equipment:**
   - **Leash:** A standard 6-foot leash is usually best for walking. Avoid retractable leashes, as they can be dangerous.
   - **Collar or Harness:** Most dogs are fine with a collar, but some may prefer a harness, especially if they pull on the leash.
   - **Poop Bags:** Always carry bags to clean up after your 

In [17]:
# This code sets up a Pinecone client, connects to an index, and performs a similarity query to retrieve the most relevant stored vectors for a given input.
# Returns nothing because there are no matches given the query criteria
os.environ["PINECONE_API_KEY"] = PINECONE_API_TOKEN
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index = pc.Index(host="https://sample-movies-h879cws.svc.aped-4627-b74a.pinecone.io")

index.query(
    namespace="( Default )",
    vector=[0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
    top_k=3,
    include_values=True)

{'matches': [], 'namespace': '( Default )', 'usage': {'read_units': 1}}

In [18]:
# Creates new index with a specific dimensions in Pinecone
index_name = "my-valid-index"
index_dimension = 1536

# Check if the index exists and create it if not
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=index_dimension,
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [ ]:
# Connect to the index
index = pc.Index(name=index_name)

# Insert data into the index
vectors = [
    {"id": "vec1", "values": [0.1] * index_dimension},
    {"id": "vec2", "values": [0.4] * index_dimension}
]

index.upsert(vectors)

# Query the index using keyword arguments
query_results = index.query(
    vector=[0.1] * index_dimension,
    top_k=5
)

print(query_results)

In [19]:
# Connecting to the pre-populated pinecone Index
index = pc.Index('sample-movies')

# Replace 384 with the actual dimension of your embeddings
query_vector = np.random.rand(384).tolist()

# Query the index
response = index.query(
    vector=query_vector,
    top_k=5,  # Number of nearest neighbors to retrieve
    include_values=True,  # Include the vector values in the response
    include_metadata=True,  # Include metadata like movie titles, genres, etc.
    namespace="default"  # Use the default namespace or your specific namespace
)

# Process the response
for match in response.matches:
    print(f"ID: {match.id}, Score: {match.score}, Metadata: {match.metadata}")

In [20]:
indexes = pc.list_indexes()
print("Available indexes:", indexes)

Available indexes: [{
    "name": "my-valid-index",
    "metric": "euclidean",
    "host": "my-valid-index-h879cws.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}, {
    "name": "sample-movies",
    "metric": "cosine",
    "host": "sample-movies-h879cws.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null
}]


In [21]:
# Trying to fit the dimensions
index_dimension = 1024
index_name = "sample-movies"
index = pc.Index(name=index_name)

In [22]:
# Example found on the Pinecone index
vector_sample = 0.0234527588, 0.0291595459, -0.0386352539, -0.0106735229, 0.0202941895, -0.0122756958, 0.00450897217, 0.0818481445, 0.0360717773, -0.0252075195, 0.0346984863, 0.0227661133, -0.0592956543, -0.0495605469, -0.00486373901, 0.01171875, -0.0222167969, 0.0142593384, 0.000259876251, -0.0293884277, 0.0318908691, -0.0526123047, -0.075378418, -0.0262298584, 0.0000813603401, -0.0197296143, -0.00490188599, -0.0194091797, -0.0264282227, -0.0335083, -0.0387878418, 0.00572967529, -0.0230407715, -0.018157959, -0.00319671631, 0.0283355713, 0.0467224121, 0.022277832, -0.0512084961, 0.0535583496, -0.0127563477, 0.0238952637, -0.0602417, -0.0192871094, -0.0371704102, 0.0144348145, 0.008644104, 0.0104599, -0.0196075439, 0.0320129395, 0.0099029541, 0.00342941284, -0.011680603, 0.0004093647, -0.0338134766, 0.018371582, -0.015838623, 0.0317382812, -0.0770874, -0.00546264648, -0.00706863403, 0.00489807129, -0.00102424622, -0.0121383667, -0.0237884521, 0.0540161133, 0.0038318634, 0.000693321228, -0.0487670898, 0.0480957031, -0.0107879639, 0.0332641602, -0.0102310181, -0.0123062134, -0.0126876831, -0.0561218262, 0.0435791, -0.0132598877, 0.00524139404, -0.0107345581, 0.095703125, -0.0273284912, -0.00345802307, -0.0173492432, 0.0162658691, 0.0384521484, 0.0284881592, 0.0246124268, 0.0326538086, 0.0643310547, 0.0261077881, 0.0462036133, 0.011138916, -0.0296478271, -0.0791015625, -0.00335884094, 0.0457763672, 0.018951416, 0.00548172, 0.0132522583, 0.0123062134, -0.0404663086, 0.0160675049, -0.0440979, -0.0308380127, 0.0462036133, 0.00887298584, 0.0106201172, 0.0528869629, 0.00587463379, 0.0602111816, 0.0497741699, 0.0245361328, -0.0254974365, -0.0301055908, 0.0224304199, 0.0195007324, -0.0314025879, 0.0111923218, 0.02684021, 0.0366210938, 0.0416564941, -0.020111084, -0.020690918, 0.008644104, -0.0254058838, -0.00820159912, 0.0466308594, -0.00848388672, 0.0549316406, 0.0260925293, -0.0232391357, -0.0324401855, -0.00975036621, -0.032043457, -0.00212669373, -0.0107650757, -0.0264434814, -0.0352478027, 0.00481033325, -0.00295829773, 0.0484008789, 0.0447387695, -0.0286254883, -0.0374450684, -0.0608825684, -0.0415039062, 0.0307006836, -0.00775146484, -0.0245666504, -0.000881195068, -0.0287780762, -0.0221099854, 0.015335083, 0.038482666, -0.00116539, 0.0335388184, 0.0246124268, 0.00518798828, 0.0284118652, -0.037902832, -0.0760498047, -0.0204162598, -0.0323181152, 0.00533294678, -0.0148010254, 0.0476074219, 0.0268249512, 0.024307251, -0.0405578613, -0.0186004639, -0.0517578125, -0.0125808716, -0.0483093262, 0.00389671326, 0.0132751465, 0.01612854, -0.00163269043, 0.0255432129, -0.0464172363, -0.0673217773, -0.00770950317, 0.017364502, -0.0166931152, -0.0176086426, 0.00960540771, 0.0184631348, 0.0134429932, 0.0291900635, 0.0315551758, 0.0321350098, -0.0268249512, -0.00119400024, 0.0165710449, -0.0222473145, 0.0220947266, 0.0339355469, 0.00566101074, -0.00648498535, -0.0272827148, -0.010925293, 0.0321655273, -0.0496521, 0.00147914886, 0.00371551514, -0.0454101562, -0.01197052, -0.0451049805, 0.0166320801, -0.0407409668, -0.0289459229, 0.00541687, 0.0194854736, -0.0149917603, -0.0623168945, -0.0798339844, -0.0267181396, -0.00901794434, 0.0301818848, -0.0218048096, 0.00142669678, 0.0172424316, 0.00662612915, 0.0335388184, 0.0106201172, 0.0191345215, 0.0501403809, 0.0279541016, 0.030166626, 0.00797271729, 0.0124053955, 0.0137329102, -0.0501709, 0.0283050537, 0.0384521484, -0.0162200928, -0.0359191895, -0.00430297852, 0.0477905273, 0.0398254395, -0.0294952393, 0.0301818848, 0.00534820557, -0.00416564941, 0.0190887451, -0.0184021, 0.0140151978, -0.0274658203, 0.0170440674, -0.0199432373, 0.0426330566, 0.0575256348, 0.0085144043, -0.0262908936, -0.00149250031, 0.0233764648, 0.00111675262, 0.0260314941, -0.0140838623, -0.0197296143, -0.00120353699, 0.0202789307, 0.0142059326, 0.0398254395, 0.0347595215, 0.00176620483, 0.0105056763, 0.03515625, -0.0198669434, -0.0298309326, -0.00300216675, -0.023651123, -0.0371704102, -0.0118103027, -0.067199707, 0.0306854248, -0.0140609741, -0.015914917, -0.0302429199, 0.0270996094, 0.00313186646, -0.00702285767, 0.0187835693, -0.0207214355, 0.00746536255, -0.0323181152, -0.00357627869, -0.0242919922, -0.00133895874, -0.0111083984, 0.0416870117, -0.00378036499, 0.0505065918, 0.0407409668, 0.0556030273, 0.0483093262, -0.0245361328, -0.0758667, -0.0434570312, 0.0300292969, -0.0484008789, -0.0631713867, 0.02684021, 0.0613098145, -0.00925445557, -0.0234985352, -0.0225982666, -0.0173187256, -0.0504760742, -0.0129089355, 0.0174560547, 0.0722045898, 0.0072593689, 0.00225830078, -0.0562438965, 0.00661468506, -0.0500488281, 0.00689315796, -0.00321388245, 0.052734375, -0.0371704102, 0.0078086853, -0.0270690918, 0.0064125061, 0.0175018311, -0.00349235535, 0.0202789307, -0.022567749, 0.013999939, -0.0126495361, 0.00694656372, -0.0344848633, -0.00454330444, -0.0092086792, -0.0249328613, 0.0285491943, 0.0150299072, 0.0162658691, 0.0277557373, 0.0060043335, 0.0165405273, -0.0497741699, -0.0599365234, -0.0220489502, 0.0247497559, -0.049407959, 0.034362793, -0.022644043, 0.0521850586, 0.0166168213, 0.0207672119, -0.0304870605, 0.00505828857, 0.033782959, -0.0191497803, 0.114990234, -0.0116424561, -0.0141906738, 0.031463623, -0.012512207, -0.00104999542, 0.0072517395, 0.0455627441, -0.024230957, -0.0286560059, -0.00904846191, -0.00944519, -0.0456237793, -0.0244140625, -0.0454101562, 0.00386810303, 0.0261077881, 0.00833892822, 0.0125579834, 0.00925445557, -0.0187072754, -0.0483093262, -0.0137863159, 0.0200500488, -0.0472717285, -0.0198669434, -0.0364074707, 0.0133972168, -0.0327148438, 0.00937652588, -0.0654296875, 0.212402344, 0.00730896, 0.00773239136, -0.0331420898, -0.00408172607, 0.00788879395, 0.00888061523, 0.0150527954, 0.0188293457, 0.0573730469, -0.0244445801, -0.0286712646, 0.0194549561, -0.0147399902, -0.00929260254, 0.0283508301, 0.00534820557, 0.0407104492, 0.0372924805, 0.0188751221, 0.0592651367, -0.000204563141, 0.0436096191, 0.0270233154, -0.0289611816, -0.0472412109, -0.00657272339, 0.0371398926, -0.0393066406, 0.017578125, -0.0428161621, 0.0093536377, -0.00845336914, 0.0133972168, -0.0161590576, 0.0015668869, 0.0202026367, -0.0293426514, 0.020690918, -0.0462341309, -0.0082321167, -0.0301971436, -0.0522155762, 0.0869751, -0.0071105957, 0.0227966309, 0.0299224854, -0.0413818359, -0.0536499023, 0.00963592529, 0.00936126709, 0.00145721436, -0.0239868164, -0.0166320801, -0.00391769409, -0.0661010742, -0.0469055176, -0.040435791, 0.0191497803, 0.0281677246, 0.0749511719, 0.0125579834, -0.0619506836, -0.0500183105, 0.0444335938, 0.0303955078, 0.000474691391, -0.0412597656, -0.0167694092, -0.0268707275, -0.0598449707, -0.0434570312, -0.0342102051, 0.0280303955, 0.00103759766, -0.0173492432, -0.0456542969, 0.0328063965, 0.0192108154, -0.008644104, 0.0521850586, 0.00624847412, -0.0422668457, -0.0169677734, 0.0352478027, 0.00152015686, 0.0244140625, 0.00173473358, -0.0158844, 0.0148696899, 0.0285491943, 0.0408630371, -0.0168304443, 0.0620727539, 0.0158081055, -0.0497131348, 0.0018787384, 0.029876709, -0.0191497803, -0.0299682617, 0.00119686127, 0.0213165283, -0.0282287598, -0.0740356445, -0.0248718262, -0.0293121338, 0.0365905762, 0.0390930176, 0.0244140625, -0.0167541504, 0.0187072754, -0.0205230713, 0.00401687622, 0.00737762451, 0.0414428711, 0.0232849121, -0.00445556641, -0.0450439453, 0.025970459, 0.0115966797, 0.00675582886, 0.0333557129, -0.031463623, 0.0430297852, -0.0023021698, 0.0415344238, -0.0552063, 0.0373535156, 0.0452575684, 0.0225372314, 0.0207672119, -0.0213775635, -0.0500488281, 0.0112075806, 0.0120315552, 0.00337219238, 0.0420227051, -0.03125, 0.00975799561, 0.0360412598, -0.0192718506, 0.00570678711, -0.061920166, 0.0447387695, 0.00900268555, -0.00577163696, -0.0166320801, -0.0407104492, -0.0171356201, -0.00744628906, 0.0187988281, -0.0371704102, 0.0140151978, 0.0234985352, -0.0323791504, 0.00834655762, -0.000173330307, -0.00549316406, 0.0221405029, -0.0512695312, -0.0307159424, -0.0243225098, 0.0170440674, 0.102172852, 0.0464477539, 0.0648803711, -0.0220794678, 0.0272827148, 0.0133209229, 0.0230560303, -0.0187835693, -0.0407104492, 0.0299377441, 0.0704345703, 0.0476989746, -0.0172424316, -0.0150527954, 0.0268249512, -0.0418701172, 0.00591278076, -0.0135879517, 0.0367736816, 0.024017334, -0.0601501465, 0.00835418701, 0.000427246094, 0.00785827637, -0.0225524902, -0.0597839355, 0.00295066833, 0.00659179688, 0.0497436523, -0.0166015625, -0.0280761719, -0.0183105469, -0.0112762451, 0.0144424438, -0.00437927246, 0.00310134888, -0.0270690918, 0.00594329834, -0.0379638672, -0.0134353638, 0.0165557861, -0.00221824646, -0.0174560547, 0.0051574707, 0.0244140625, 0.0290222168, 0.0176239014, 0.00309944153, -0.0355529785, -0.0105514526, 0.0203552246, 0.0331115723, 0.0159454346, 0.0516052246, 0.015838623, 0.0320129395, -0.0425720215, -0.0147323608, 0.0419616699, 0.0258026123, -0.0348205566, -0.0175018311, -0.00508880615, 0.0296936035, 0.021774292, -0.00977325439, -0.00513839722, 0.00680923462, -0.0123977661, -0.0474243164, -0.047454834, -0.0322875977, -0.0256500244, 0.0368347168, 0.0157775879, 0.0110244751, -0.00236701965, 0.0197906494, -0.00605011, -0.014251709, -0.0219116211, -0.0137329102, 0.0243988037, -0.0466003418, -0.0249176025, 0.0337524414, 0.0576477051, -0.0103149414, -0.0370788574, 0.0362548828, 0.0395507812, -0.0450439453, -0.059387207, 0.00774765, -0.0120773315, -0.0227661133, -0.0455627441, 0.0340271, -0.00160884857, -0.00105190277, 0.013130188, -0.0523681641, 0.0572814941, 0.0469970703, -0.0438232422, -0.0138320923, -0.0245056152, 0.0144119263, -0.00290679932, -0.0460205078, -0.0601196289, 0.0532836914, 0.0302581787, -0.0268707275, -0.0387878418, -0.000377893448, -0.0176544189, -0.0586547852, 0.0426940918, 0.003490448, 0.00814056396, 0.0354309082, 0.0553894043, -0.0595703125, 0.0147094727, -0.0168304443, 0.000684738159, -0.0130233765, 0.0375061035, -0.0176849365, 0.01953125, -0.0537109375, -0.067565918, -0.0366516113, 0.00074338913, 0.0110092163, -0.0389709473, 0.0143585205, -0.0155715942, -0.0331420898, -0.00824737549, 0.008644104, 0.0416564941, -0.0280609131, -0.0238037109, 0.039276123, 0.00780487061, 0.0177612305, -0.0311737061, 0.0266265869, 0.0420532227, -0.00273132324, 0.00736999512, -0.0234680176, -0.0380554199, -0.0233001709, 0.033782959, -0.0191040039, -0.0397644043, -0.0156021118, 0.0206604, 0.0278015137, 0.0044631958, 0.0102462769, 0.0179290771, 0.0229034424, -0.0213470459, 0.0516357422, 0.00162124634, -0.0326538086, -0.00107955933, -0.0424804688, -0.0237884521, 0.00191307068, -0.0385437, -0.0116882324, -0.0327758789, -0.0247039795, 0.0352172852, 0.0233764648, -0.0480651855, 0.0166015625, -0.0122756958, 0.0137329102, 0.0145263672, 0.00855255127, -0.00331115723, -0.0134887695, 0.0355529785, 0.0148544312, 0.00044131279, -0.046081543, 0.0385131836, -0.043182373, 0.0141296387, 0.0221252441, -0.0222167969, -0.0277252197, 0.00190258026, 0.0331115723, -0.0064163208, -0.00804138184, 0.0491638184, 0.00907135, 0.0388183594, -0.00443267822, 0.0358276367, -0.0289459229, 0.00389099121, -0.0390930176, -0.0395813, -0.0292816162, 0.0651245117, -0.0433044434, 0.028793335, 0.0128555298, 0.0371398926, 0.00240516663, -0.0371704102, -0.0272369385, 0.0110626221, 0.0110702515, -0.0438842773, -0.00142383575, -0.00988769531, -0.0109176636, 0.041015625, -0.021484375, 0.0247802734, -0.022064209, 0.00344085693, -0.0662841797, -0.0248718262, 0.0149002075, -0.0291442871, -0.0127868652, -0.0322875977, -0.0513916, 0.0539550781, 0.0202484131, -0.0120162964, -0.0348205566, -0.0209655762, 0.00349807739, 0.000747680664, 0.00269889832, -0.00590515137, 0.0159454346, 0.00302314758, 0.0055847168, 0.0162811279, 0.0194091797, -0.0346374512, 0.00279808044, 0.000793457031, 0.0551452637, 0.0257568359, 0.0151290894, 0.022644043, -0.0314025879, -0.00891876221, -0.0112609863, -0.0183868408, -0.00153446198, 0.0326843262, -0.0595092773, 0.0321044922, -0.0287017822, -0.0291595459, -0.0393371582, 0.0201721191, -0.0454101562, 0.0526123047, -0.0108642578, 0.0281982422, -0.0187072754, 0.0502624512, -0.0145721436, -0.0069732666, 0.0211792, -0.0131073, -0.0163879395, -0.0531005859, 0.0382080078, 0.0453796387, 0.0166931152, 0.0228881836, 0.0115890503, -0.0250549316, 0.061340332, 0.05078125, 0.0213470459, -0.0174865723, -0.0354309082, -0.0472106934, -0.0309906, -0.037902832, -0.0484008789, -0.0411987305, 0.00172996521, 0.00678634644, 0.0208282471, -0.05078125, -0.00562286377, 0.0188903809, 0.0239105225, -0.0102233887, -0.00946807861, 0.029083252, 0.00645446777, -0.00391769409, 0.00141429901, -0.0690918, 0.000500679, 0.026260376, -0.0031375885, 0.00205230713, -0.00184440613, -0.0309295654, -0.0372009277, 0.0287475586, 0.000243067741, 0.0345459, 0.0360107422, -0.0444946289, -0.0268249512, 0.00649642944, -0.0314941406, -0.0126342773, -0.00940704346, 0.0107574463, -0.0394592285, -0.00584793091, 0.00748062134, -0.0116653442, -0.00765609741, 0.0160522461, 0.0347290039, 0.00163173676, 0.0299072266, -0.0204315186, -0.0495605469, -0.0199432373, -0.0433349609, -0.0412902832, -0.0194091797, 0.025604248, 0.0117950439, 0.00966644287, -0.0217132568, 0.0134429932, -0.0440979, -0.0483398438, 0.0339355469, -0.0113983154, 0.00751113892, 0.00551986694, -0.0164642334, -0.034362793, -0.017791748, -0.0124816895, 0.0382385254, -0.0536499023, 0.0407409668, 0.0205993652, 0.00512313843, -0.0487365723, -0.00816345215, 0.0339355469, -0.00317573547, 0.0400085449, -0.0228729248, -0.0305938721, -0.0114593506, -0.00865936279, -0.00727462769, 0.00642776489, 0.00618743896, -0.0425720215, -0.041809082, -0.0309448242, 0.00512313843, -0.0355529785, -0.019317627, 0.00828552246, 0.0182647705, 0.010345459, 0.022277832, -0.0230712891, 0.0552368164, 0.0118713379, -0.00983429, -0.0324401855, -0.0642700195, 0.0377502441, 0.00845336914, -0.0385437, 0.0332336426, 0.0183105469, -0.0200500488, 0.00240898132, 0.0052948, 0.037902832, -0.0344238281, -0.00131988525, 0.0201873779, -0.00495529175, 0.05859375, 0.0162811279, 0.00293922424, 0.0336303711, 0.0555114746, -0.00849914551, -0.0194854736, 0.027923584, 0.0170440674, 0.0347290039, 0.00113201141, -0.0304718018, -0.0273742676, 0.0381469727, -0.0148162842, 0.0226898193, -0.00420379639, 0.0140991211, 0.0213928223, -0.0370178223, -0.0214538574, 0.0148544312, -0.0570373535, -0.0193634033, 0.0445251465, -0.00566101074, 0.00279808044, -0.00877380371, -0.0288848877, -0.0220947266, -0.034576416, 0.0242004395, -0.03515625, 0.0413818359, 0.0423584, 0.00235748291, 0.0288696289, 0.0232391357, 0.0145721436, 0.0220031738, -0.0104522705, -0.0385437, -0.00359344482, 0.0385437, 0.0447692871, 0.0538024902, 0.0108642578, -0.020111084, -0.0204315186, 0.0218200684, -0.0283355713, 0.00341033936, 0.00797271729, -0.00747680664, -0.0179595947, 0.0200500488, 0.0076751709, -0.00331878662, -0.013343811, -0.0298614502, 0.0225219727, 0.002784729, 0.0171356201, 0.0401916504, 0.00205230713, 0.00196838379, -0.0272979736, 0.00506210327

In [23]:
# Outputs responses that are similar to the example provided above
response = index.query(
    vector=vector_sample,
    top_k=20,
)

print("Sample vector query response:", response)

Sample vector query response: {'matches': [{'id': '0', 'score': 0.999857843, 'values': []},
             {'id': '2', 'score': 0.831563532, 'values': []},
             {'id': '28', 'score': 0.827061057, 'values': []},
             {'id': '9', 'score': 0.816736162, 'values': []},
             {'id': '6', 'score': 0.814963937, 'values': []},
             {'id': '5', 'score': 0.807208717, 'values': []},
             {'id': '1', 'score': 0.805884, 'values': []},
             {'id': '12', 'score': 0.800955892, 'values': []},
             {'id': '14', 'score': 0.800257862, 'values': []},
             {'id': '24', 'score': 0.798902154, 'values': []},
             {'id': '21', 'score': 0.795445, 'values': []},
             {'id': '20', 'score': 0.795321047, 'values': []},
             {'id': '3', 'score': 0.793760478, 'values': []},
             {'id': '79', 'score': 0.793173432, 'values': []},
             {'id': '11', 'score': 0.791409433, 'values': []},
             {'id': '18', 'score': 0.7

In [24]:
# Initialize embeddings, model has the dimensions of 1024 which matches the Sample Movies Index
model_name = "sentence-transformers/all-roberta-large-v1"  #
embeddings = HuggingFaceEmbeddings(model_name=model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [37]:
pip install langchain_pinecone

In [38]:
pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.8 MB/s eta 0:00:00


In [39]:
import os
import pinecone
from langchain_pinecone import Pinecone  # ✅ Correct import
from langchain_openai import OpenAIEmbeddings  # ✅ Correct import for OpenAI embeddings

# 1. Initialize Pinecone
pinecone.init(api_key=os.getenv("PINECONE_API_KEY"), environment="us-west1-gcp")  # Adjust as needed

# 2. Connect to an existing index
index_name = "your-index-name"
index = pinecone.Index(index_name)  # ✅ This ensures correct type

# 3. Define an embedding model
embeddings = OpenAIEmbeddings()

# 4. Initialize Pinecone vectorstore for LangChain
vectorstore = Pinecone(index=index, embedding=embeddings, text_key="text")  # ✅ Corrected


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [32]:
# 1. Initialize Pinecone
pinecone.init(api_key=os.getenv("PINECONE_API_KEY"), environment="us-west1-gcp")  # Adjust as needed

# 2. Connect to an existing index
index_name = "your-index-name"
index = pinecone.Index(index_name)  # ✅ This ensures correct type

# 3. Define an embedding model
embeddings = OpenAIEmbeddings()

# 4. Initialize Pinecone vectorstore for LangChain
vectorstore = Pinecone(index=index, embedding=embeddings, text_key="text")  # ✅ Corrected

NameError: name 'pinecone' is not defined

In [30]:
# Specifies which key in the metadata of stored documents will be used as the searchable content.
text_field = "summary"

# Accesses an existing Pinecone index named index_name.
index = pc.Index(index_name)

# Accesses an existing Pinecone index named index_name to create a Langchain compatible vectorstore
vectorstore = Pinecone(
    index, embeddings.embed_query, text_field
)

<ipython-input-30-1bc243a25d29>:8: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  vectorstore = Pinecone(
/usr/local/lib/python3.11/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


ValueError: client should be an instance of pinecone.Index, got <class 'pinecone.data.index.Index'>

In [26]:
query = "Cars animation rating?"

results = vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

NameError: name 'vectorstore' is not defined

In [27]:
results

NameError: name 'results' is not defined

In [28]:
for doc in results:
  print(f"title: {doc.metadata['title']} - description: {doc.page_content}: ")

NameError: name 'results' is not defined

In [ ]:
query = "Who are the main characters in movie title Zootopia?"

results = vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

for doc in results:
  print(f"title: {doc.metadata['title']} - description: {doc.page_content}: ")

title: Jurassic World - description: Set in a fully functioning dinosaur theme park on Isla Nublar, Jurassic World faces chaos when a genetically engineered dinosaur, the Indominus Rex, escapes. Stars Chris Pratt and Bryce Dallas Howard.: 
title: Star Wars: Episode III - Revenge of the Sith - description: As the Clone Wars near an end, the Sith Lord Darth Sidious steps out of the shadows, setting the stage for the creation of the Empire and Anakin Skywalker's fall to the dark side.: 
title: Inside Out - description: Young Riley's emotions - Joy, Fear, Anger, Disgust, and Sadness - conflict on how best to navigate a new city, house, and school after she moves from the Midwest to San Francisco.: 


In [ ]:
results

[Document(metadata={'box-office': 1495696292.0, 'genre': 'Drama/Romance', 'title': 'Top Gun: Maverick', 'year': 2022.0}, page_content='Capt. Pete "Maverick" Mitchell, after decades of service as one of the Navy\'s top aviators, confronts his past while training a new squad for a dangerous mission. Tom Cruise reprises his iconic role, showcasing thrilling aerial stunts.'),
 Document(metadata={'box-office': 1671537444.0, 'genre': 'Action/Adventure', 'title': 'Jurassic World', 'year': 2015.0}, page_content='Set in a fully functioning dinosaur theme park on Isla Nublar, Jurassic World faces chaos when a genetically engineered dinosaur, the Indominus Rex, escapes. Stars Chris Pratt and Bryce Dallas Howard.'),
 Document(metadata={'box-office': 868390560.0, 'genre': 'Sci-Fi/Adventure', 'title': 'Star Wars: Episode III - Revenge of the Sith', 'year': 2005.0}, page_content="As the Clone Wars near an end, the Sith Lord Darth Sidious steps out of the shadows, setting the stage for the creation of

In [ ]:
query = "Which titled star wars movie came out in 2016 and genre is Sci-Fi/Adventure?"

results = vectorstore.similarity_search(
    query,  # our search query
    k=1  # return 3 most relevant docs
)

for doc in results:
  print(f"title: {doc.metadata['title']} - description: {doc.page_content}: ")

title: Despicable Me 2 - description: Gru is recruited by the Anti-Villain League to help deal with a powerful new super criminal.: 


In [ ]:
llm = HuggingFaceHub(
    # repo_id="EleutherAI/gpt-neo-2.7B",  # Choose an appropriate model
    repo_id="distilgpt2",
    model_kwargs={"temperature": 0.7, "max_length": 200}
)

# Set up your vector store (assume you already have Pinecone index created)
# pinecone.init(api_key="YOUR_PINECONE_API_KEY", environment="us-west1-gcp")
index = pc.Index("sample-movies")

# Define the Pinecone vector store
vectorstore = Pinecone(
    index, embeddings.embed_query, text_field
)
# Define the retriever from the vector store
retriever = vectorstore.as_retriever()

# Create a QA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # You can change to other types depending on your needs
    retriever=retriever
)

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [ ]:
qa.invoke(query)

{'query': 'Which titled star wars movie came out in 2016 and genre is Sci-Fi/Adventure?',
 'result': 'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nGru is recruited by the Anti-Villain League to help deal with a powerful new super criminal.\n\nBilbo and company are forced to engage in a war against an array of combatants and keep the Lonely Mountain from falling into the hands of a rising darkness.\n\nThe final chapter of the epic trilogy where Gandalf and Aragorn lead the World of Men against Sauron\'s army to draw his gaze from Frodo and Sam\'s approach to Mount Doom.\n\nCapt. Pete "Maverick" Mitchell, after decades of service as one of the Navy\'s top aviators, confronts his past while training a new squad for a dangerous mission. Tom Cruise reprises his iconic role, showcasing thrilling aerial stunts.\n\nQuestion: Which titled star wars movie came out in 2016 a

In [ ]:
# Sample documents related to Zootopia
documents = [
    {"id": "zootopia1", "text": "Judy Hopps is a determined bunny who dreams of becoming a police officer in Zootopia.", "metadata": {"title": "Zootopia"}},
    {"id": "zootopia2", "text": "Nick Wilde is a clever fox who becomes an unexpected ally to Judy Hopps.", "metadata": {"title": "Zootopia"}},
    {"id": "zootopia3", "text": "Ginnifer Goodwin voices Judy Hopps, while Jason Bateman voices Nick Wilde in the animated film Zootopia.", "metadata": {"title": "Zootopia"}}
]

# Ensure all documents have a text key
def validate_and_add_documents(docs):
    valid_docs = [doc for doc in docs if 'text' in doc]
    if valid_docs:
        vector_store.add_documents(valid_docs)
    else:
        print("No valid documents to add")

# Add documents to the vector store
validate_and_add_documents(documents)


In [ ]:
from langchain_core.documents import Document

In [ ]:
import numpy as np

def pad_vector(vector, target_dim=1024):
    current_dim = len(vector)
    if current_dim < target_dim:
        padding = [0] * (target_dim - current_dim)
        return vector + padding  # Pad the vector with zeros
    return vector  # Already the correct dimension

# Example resizing
original_vector = np.random.rand(768).tolist()  # Example 768-dimensional vector
padded_vector = pad_vector(original_vector, target_dim=1024)
print("Padded vector:", padded_vector)
print(len(padded_vector))


Padded vector: [0.8999735483814706, 0.7007996739531873, 0.23377873373307656, 0.7868123354218262, 0.7823527686485777, 0.4380837889873457, 0.5198942810532384, 0.2551334824196595, 0.028269477770443974, 0.7097181856375856, 0.06701164635255852, 0.17093968422751327, 0.7881207370724994, 0.08415104846982857, 0.5516197983578566, 0.9868247996399457, 0.17801254936838928, 0.4357539510262556, 0.2384112399617776, 0.5293087848873919, 0.06421098474283815, 0.22764842755884673, 0.7057193910291404, 0.9324331684133014, 0.8580595040626448, 0.39459514544983953, 0.8990606791204963, 0.42605179846310504, 0.24014603618029595, 0.29988777214008855, 0.5999329693397658, 0.1965651138572485, 0.1811036207979917, 0.8684151208954225, 0.4857126994849835, 0.27709479797972025, 0.8667317750696355, 0.528901988021621, 0.036411472568207426, 0.4538186830448432, 0.21951152024694076, 0.9712522505802191, 0.1958107755073728, 0.623684909232793, 0.3708980807814626, 0.06680575245227738, 0.6425918693033579, 0.9168018120553197, 0.180190

Looks like the issue with the langchain RAG APP

Embedding Issues\
The embedding for the stored documents might not be aligned with the embedding used to process the query. For example, the documents were embedded with one model (e.g., sentence-transformers/all-MiniLM-L6-v2) and the query is being embedded using a different model or function.

Irrelevant Document Content\
If the document content in the Pinecone index is too general or unrelated to the query (e.g., not covering Star Wars movies), the similarity search might return irrelevant results.

Index Configuration\
The Pinecone index may not have been correctly configured for semantic similarity search. For example: Incorrect vector dimensions, missing fields in metadata or inadequate number of indexed documents.

Document Metadata Issue\
The metadata field (title) you are trying to access might not exist in the indexed document.
